## Custom Maxwell Solvers
Our FDFD Maxwell solver is great for simple tasks. The solver does not need to be fast: to compute limits, you only need to compute the inverse of the background Green's function mapping currents in the design region to fields in the design region, *once*. 

Still, if you want to slot in your own Maxwell solver, this tutorial will cover how to do so. This might help have better numerical consistency between the limits and your own inverse designs, or allow you to calculate limits for more complex systems.

In [3]:
import inspect
from dolphindes import photonics, maxwell, geometry

In [5]:
# The photonics class has an attribute, self.EM_solver, which it calls to compute electromagnetic quantities. This is calculated in setup_EM_solver():
print(inspect.getsource(photonics.Photonics_TM_FDFD.setup_EM_solver))

    def setup_EM_solver(
        self, geometry: Optional[GeometryHyperparameters] = None
    ) -> None:
        """
        Set up the FDFD electromagnetic solver with given geometry.

        Parameters
        ----------
        geometry : CartesianFDFDGeometry or PolarFDFDGeometry
            Geometry specification. If None, uses self.geometry.

        Notes
        -----
        Creates a TM_FDFD or TM_Polar_FDFD solver instance and stores it in
        self.EM_solver.
        """
        if geometry is not None:
            self.geometry = geometry

        assert self.geometry is not None
        if isinstance(self.geometry, CartesianFDFDGeometry):
            self._flatten_order = "C"
            check_attributes(
                self.geometry,
                "Nx",
                "Ny",
                "Npmlx",
                "Npmly",
                "dx",
                "dy",
                "bloch_x",
                "bloch_y",
            )
            self.EM_solver = T

In [7]:
# TM_FDFD is our TM FDFD code. All you have to do is over-write this function with your own custom solver. i.e. 

class custom_maxwell_photonics(photonics.Photonics_TM_FDFD):
    def setup_EM_solver(self, geometry):
        # Do whatever you want with the parameters 
        self.EM_solver = None # here replace your custom solver
        pass 

In [8]:
# What should the EM_solver know what to do? 
# At minimum, it should be able to compute the off-diagonal block of the inverse of the Green's function which corresponds to design -> design. This gets called in setup_EM_operators, which is needed for setting up the QCQP for limit calculations. 
# (This may look different once everything has been implemented)
print(inspect.getsource(photonics.Photonics_TM_FDFD.setup_EM_operators))

    def setup_EM_operators(self) -> None:
        """
        Set up electromagnetic operators for the design region and background.

        Notes
        -----
        This method creates the appropriate operators based on whether sparse or dense
        QCQP formulation is used:
        - For sparse QCQP: Creates Ginv (inverse Green's function) and M operators
        - For dense QCQP: Creates G (Green's function) operator

        Requires self.des_mask to be defined.

        Raises
        ------
        AttributeError
            If des_mask is not defined.
        """
        check_attributes(self, "des_mask")
        assert self.EM_solver is not None
        if self.sparseQCQP:
            self.Ginv, self.M = self.EM_solver.get_GaaInv(
                self.des_mask, self.chi_background
            )
        else:
            if self.chi_background is None:
                self.M = self.EM_solver.M0
                if isinstance(self.geometry, CartesianFDFDGeometry):
          

In [8]:
# For reference, this is what our get_GaaInv looks like:
print(inspect.getsource(maxwell.TM_FDFD.get_GaaInv))

    def get_GaaInv(self, A_mask: np.ndarray, chigrid: np.ndarray = None) -> tuple[np.ndarray, sp.csc_array]:
        """
        Compute the inverse Green’s function on region A, G_{AA}^{-1}, using a Woodbury identity.

        We partition the full Maxwell operator M into blocks corresponding to region A (design)
        and its complement B (background):
            M = [[A, B],
                 [C, D]]
        Then G_{AA}^{-1} = D - C A^{-1} B, up to a multiplicative constant MU_0 / k^2.

        Parameters
        ----------
        A_mask : np.ndarray of bool, shape (Nx, Ny)
            Mask for the design region A.
        chigrid : np.ndarray of complex, optional
            Material susceptibility distribution. If provided, M = M0 + diag(ω² χ).

        Returns
        -------
        GaaInv : sp.csc_array of shape (n_A, n_A)
            The inverse Green’s function on region A.
        M : sp.csc_array
            The full Maxwell operator used in the computation.
        """


In [9]:
# What else should the EM_solver know what to do? If you plan to pass an incident current, it needs to know how to calculate fields from the currents. This is called in get_ei()
print(inspect.getsource(photonics.Photonics_TM_FDFD.get_ei))
# Alternatively, you can just pass ji = None and set_ei with a known incident field of size Nx, Ny via the attribute self.ei or by calling set_ei()

    def get_ei(
        self, ji: Optional[ComplexGrid] = None, update: bool = False
    ) -> ComplexArray:
        """
        Return the incident field.

        Parameters
        ----------
        ji : ndarray of complex, optional
            Current source.
        update : bool, optional
            Whether to update stored incident field. Default: False

        Returns
        -------
        ndarray of complex
            Incident electromagnetic field.
        """
        assert self.EM_solver is not None
        ei: ComplexArray
        if self.ei is None:
            assert ji is not None or self.ji is not None, (
                "Either ji argument or self.ji must be specified to compute ei."
            )
            source = ji if ji is not None else self.ji
            ei = self.EM_solver.get_TM_field(source, self.chi_background)
        else:
            ei = self.ei
        if update:
            self.ei = ei
        return ei

